# Example 1

In [1]:
#coding=utf-8

# example code from https://en.wikipedia.org/wiki/Viterbi_algorithm

def viterbi(obs, states, start_p, trans_p, emit_p):
    '''
    Consider a village where all villagers are either healthy or have a fever 
    and only the village doctor can determine whether each has a fever. 

    The doctor diagnoses fever by asking patients how they feel. 
    The villagers may only answer that they feel normal, dizzy, or cold.

    The doctor believes that the health condition of his patients 
    operate as a discrete Markov chain. There are two states, 
    "Healthy" and "Fever", but the doctor cannot observe them directly, 
    they are hidden from him. 

    On each day, there is a certain chance that the patient will tell
    the doctor he/she is "normal", "cold", or "dizzy", depending on her health condition.

    - start_probability
    represents the doctor's belief about which state the HMM is in 
    when the patient first visits (all he knows is that the patient tends to be healthy). 
    The particular probability distribution used here is not the equilibrium one, 
    which is (given the transition probabilities) approximately {'Healthy': 0.57, 'Fever': 0.43}. 

    - transition_probability
    represents the change of the health condition in the underlying Markov chain. 
    In this example, there is only a 0.3 chance that tomorrow the patient will 
    have a fever if he is healthy today.

    - emission_probability
    represents how likely the patient is to feel on each day. 
    If he is healthy, there is a 0.5 chance that he feels normal; 
    if he has a fever, there is a 0.6 chance that he feels dizzy.

    - observations 
    ('normal', 'cold', 'dizzy')

    - question
    what is the hidden state for the observations?
    '''
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]

    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}

        for y in states:
            (prob, state) = max([(V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states])
            V[t][y] = prob
            newpath[y] = path[state] + [y]

        # Don't need to remember the old paths
        path = newpath

    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])

    print "{:>7s}{}".format('index', ':'),
    for i in range(len(V)): print "{:>7d}".format(i),
    print

    print "{:>7s}{}".format('observ', ':'),
    for s in observations:
        print '{:>7s}'.format(s),
    print
    print

    for y in V[0].keys():
        print '{:>7}{}'.format(y,':'),
        for t in range(len(V)):
            print '{:.5f}'.format(V[t][y]),
        print
    print

    print "{:>7s}{}".format('state', ':'),
    for s in path[state]:
        print '{:>7s}'.format(s),
    print


if __name__ == "__main__":

    states = ('Healthy', 'Fever')

    observations = ('normal', 'cold', 'dizzy')

    start_probability = {'Healthy': 0.6, 'Fever': 0.4}

    transition_probability = {
        'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
        'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
        }

    emission_probability = {
        'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
        'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
        }
    
    viterbi(
        observations,
        states,
        start_probability,
        transition_probability,
        emission_probability)

  index:       0       1       2
 observ:  normal    cold   dizzy

Healthy: 0.30000 0.08400 0.00588
  Fever: 0.04000 0.02700 0.01512

  state: Healthy Healthy   Fever


# Example 2

In [2]:
#coding=utf-8
from __future__ import division
import numpy as np
from hmmlearn import hmm

'''
Consider two friends, Alice and Bob, who live far apart from each other
and who talk together daily over the telephone about what they did that day.

Bob is only interested in three activities: walking in the park, shopping, 
and cleaning his apartment. The choice of what to do is determined 
exclusively by the weather on a given day. 

Alice has no definite information about the weather where Bob lives, 
but she knows general trends. Based on what Bob tells her he did each day, 
Alice tries to guess what the weather must have been like.

Alice believes that the weather operates as a discrete Markov chain. 
There are two states, "Rainy" and "Sunny", but she cannot observe them 
directly, that is, they are hidden from her. 

On each day, there is a certain chance that Bob will perform one of 
the following activities, depending on the weather: "walk", "shop", or
"clean". 

Since Bob tells Alice about his activities, those are the observations. 
The entire system is that of a hidden Markov model (HMM).

Alice knows the general weather trends in the area, and what Bob likes 
to do on average. In other words, the parameters of the HMM are known.

- start_probability:
represents Alice's belief about which state the HMM is in when Bob first 
calls her (all she knows is that it tends to be rainy on average). 
The particular probability distribution used here is not 
the equilibrium one, which is (given the transition probabilities) 
approximately {'Rainy': 0.57, 'Sunny': 0.43}. 

- transition_probability 
represents the change of the weather in the underlying Markov chain. 
In this example, there is only a 0.3 chance that tomorrow will be sunny 
if today is rainy. 

- emission_probability
represents how likely Bob is to perform a certain activity on each day. 
If it is rainy, there is a 0.5 chance that he is cleaning his apartment; 
if it is sunny, there is a 0.6 chance that he is outside for a walk.

observation: (0, 2, 1, 1, 2, 0)
Alice talks to Bob six days in a row and discovers that on the first day 
he went for a walk, on the second day he cleaned his apartment, on the 
third and fourth days he went shopping, on the fifth day he cleaned 
his apartment, and on the sixth day he went for a walk.

Alice's question: 
what is the most likely sequence of rainy/sunny days that would explain 
these observations? 

Answer:
This is answered by the Viterbi algorithm.
'''

states = ["Rainy", "Sunny"]
n_states = len(states)

observations = ["walk", "shop", "clean"]
n_observations = len(observations)

start_probability = np.array([0.6, 0.4])

transition_probability = np.array([
  [0.7, 0.3],
  [0.4, 0.6]
])

emission_probability = np.array([
  [0.1, 0.4, 0.5],
  [0.6, 0.3, 0.1]
])

randNum = None
model = hmm.MultinomialHMM(n_components=n_states, random_state=randNum)
model.startprob_=start_probability
model.transmat_=transition_probability
model.emissionprob_ = emission_probability

# predict a sequence of hidden states based on visible states
# predict weather based on what bob has said
bob_say = [0, 2, 1, 1, 2, 0]
bob_says = np.array([[0, 2, 1, 1, 2, 0]]).T
model.fit(bob_says)

# Find most likely state sequence corresponding to X.
logprob, alice_hears = model.decode(bob_says, algorithm="viterbi")

In [3]:
print "Bob says:", ", ".join(map(lambda x: observations[x], bob_say))
print "Alice hears:", ", ".join(map(lambda x: states[x], alice_hears))

Bob says: walk, clean, shop, shop, clean, walk
Alice hears: Sunny, Rainy, Rainy, Rainy, Rainy, Sunny


In [4]:
print "\ndecode output:"
print alice_hears
print logprob


decode output:
[1 0 0 0 0 1]
-5.84453991035


In [5]:
aaa = model.predict(bob_says)
print '\npredict output:'
print aaa


predict output:
[1 0 0 0 0 1]


In [6]:
# Compute the log probability under the model.
sss = model.score(bob_says)
print '\nscore output:'
print sss


score output:
-5.49888303332


In [7]:
# Compute the posterior probability for each state in the model.
ppp = model.predict_proba(bob_says)
print '\npredict_proba output:'
print ppp


predict_proba output:
[[  1.25132530e-11   1.00000000e+00]
 [  8.62153164e-01   1.37846836e-01]
 [  9.98896327e-01   1.10367280e-03]
 [  9.98896393e-01   1.10360690e-03]
 [  8.62702111e-01   1.37297889e-01]
 [  4.57569803e-02   9.54243020e-01]]
